In [17]:
"""
All rights can reffer to the LICENSE.md.

Created on July 19, 2018.

This module provides the several classes of autoencoder series.
The use could simply use these API without defining the structure by oneself.

@author: steven.cy.chuang
"""

import os
from time import time
import keras
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, model_from_json
from keras import backend as K
from keras.metrics import *

# Common function

In [ ]:
def load(path_folder):
    """
    The method is to save the models of autoencoders as several hdf5 files in a given path of the folder.
    Args:
        path_folder (string): the given path of the folder where contains encoder, decoder, and autoencoder
    Returns:
        encoder (keras model): the model of encoder
        decoder (keras model): the model of decoder
        autoencoder (keras model) : the model of autoencoder. autoencoder.predict(x) is equivalent to decoder.predict(encoder.predict(x))
    """
    encoder = keras.models.load_model(path_folder+"/encoder.h5")
    decoder = keras.models.load_model(path_folder+"/decoder.h5")

    with open(path_folder+"/configAutoencoder.json", "r") as jsonFile:                              
        jsonConfig = jsonFile.readlines()[0]
    autoencoder = model_from_json(jsonConfig)
    autoencoder.load_weights(path_folder+"/weightAutoencoder.h5")
    return encoder, decoder, autoencoder

# Autoencoder

In [18]:
class AE():
    def __init__(self, 
                 dim_input,  dim_latent=2,
                 lay_den_enc=[64], lay_den_dec=None, act_dense="leaky_relu", dropout_dense=0.5,
                 batch_norm=True):
        """
        The basic properties and pipeline will be defined in the initialization.
        It should be noted that lay_den_enc defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16] and plus the latent 2 means that the nodes of encoder is [64, 16, 2]. 
        Meanwhile decoder will be [2, 16, 64]. 
        Args:
            dim_input (int): the number of input dimension. All features are flatten as a vector.
            dim_latent (in): the number of the dimension for latent feature. Default is 2.
            lay_den_enc (list[int]): the numbers of each dense layer of encoder. Default is [64].
            lay_den_dec (list[int]): the numbers of each dense layer of decoder. Default is None that means reverse order of encoder.
            act_dense (string): the activation function. Default is "leaky_relu".
            dropout_dense (float): the dropout layer. Default is 0.5.
            batch_norm (bool): determine if apply batch normalization after dense layers. Default is True and epsilon=1e-5.
        """
        
        # Initialize some setting 
        self._dim_input = dim_input # all features are flatten as a vector
        self._inputs = Input(shape=(dim_input,)) 
        self._dim_latent = dim_latent
        if lay_den_dec is None:
            lay_den_dec = lay_den_enc[::-1]
        
        self._encoding(lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        self._decoding(lay_den_dec, batch_norm, act_dense, dropout_dense)
        
        self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)), name="autoencoder")

        
    def _stack_dense(self, x, lay_dense, batch_norm, act_dense, dropout_dense):
        """
        Stacking for dense layers whether encoder or decoder.
        The sequence is:
            Dense layers
            Batch normalization
            Activation function
            Dropout
        """
        for num_node in lay_dense:
            x = Dense(num_node)(x)
            if batch_norm :
                x = BatchNormalization(epsilon=1e-5)(x)
            if act_dense == "leaky_relu":
                x = LeakyReLU()(x)
            else:
                x = Activation(act_dense)(x)
            if dropout_dense > 0:
                x = Dropout(0.5)(x)
        return x

    
    def _encoding(self, lay_den_enc, batch_norm, act_dense, dropout_dense):
        """ 
        The flow of encoding
        """
        x = self._inputs

        # Stack of Dense layers
        x = self._stack_dense(x, lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        # Construct the latent as the output and build the encorder pipeline
        z = Dense(self._dim_latent)(x)
        self.encoder = Model(self._inputs, z, name="encoder")

        
    def _decoding(self, lay_den_dec, batch_norm, act_dense, dropout_dense):
        """ 
        The flow of decoding
        """
        # Build the Decoder Model
        input_latent = Input(shape=(self._dim_latent,), name="decoder_input")
        x = input_latent
        
        # Stack of Dense layers
        x = self._stack_dense(x, lay_den_dec, batch_norm, act_dense, dropout_dense)
            
        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Dense(self._dim_input, activation="sigmoid", name="decoder_output")(x)
        self.decoder = Model(input_latent, outputs, name="decoder")

        
    def fit(self,
            x_train, x_valid,
            num_epochs=50, size_batch=32, name_optim="adam", metrics=None, verb=1,
            path_temp_best=None, patience=3):
        """
        The method is for training process. 
        The users can call this method easily just putting training and validation datasets.
        The dimension of dataset is determined by [#instance, *dimInput].
        For example, dimInput is flatten as a number and the dimension of dataset is [#instance, #feature]
        If dimInput is a list to represent [width, height, channels], the dimension of dataset is [#instance, width, height, channels]
        Args:
            x_train (numpy ndarray): the training dataset.
            x_valid (numpy ndarray): the validation dataset.
            num_epochs (int): the maximal epochs for training. Default is 50.
            size_batch (int): the batch size. Default is 32.
            name_optim (string): the method for optimization. Default is adam.
            metrics (list(string or keras metrics)): the usage is the same with keras metrics for compile.
            verb(int): verbose; it is applied for both of fit and callback. The setting is similar to keras.
            path_temp_best (string): the temperory path of the best model for early-stop. Default None means without early-stop. 
            patience (int): the times of epochs to allow further trying if current loss is not better than the best. 
        Returns:
            history (keras.callbacks.History): the learning curving for the training process
            time_train (float): the consuming time of the training 
        """
        self.autoencoder.compile(optimizer=name_optim, loss="binary_crossentropy", metrics=metrics)

        if path_temp_best is None:
            callbacks = None
        else:
            if not os.path.exists(path_temp_best): # make sure the folder exists
                os.makedirs(path_temp_best)
            
            name_temp = "AutoEncoder" + str(time()) # use timestamp as unique name
            cb_es = EarlyStopping(monitor="val_loss", patience=patience, verbose=verb, mode="auto")
            chkpt = path_temp_best + "/" + name_temp + ".hdf5"
            cb_cp = ModelCheckpoint(filepath = chkpt, monitor="val_loss", verbose=verb, save_best_only=True, mode="auto")
            callbacks = [cb_es, cb_cp]
        
        # Train the autoencoder
        tic = time()
        history = self.autoencoder.fit(x_train, x_train,
                                       epochs=num_epochs,
                                       batch_size=size_batch, shuffle=True,
                                       callbacks=callbacks,
                                       validation_data=(x_valid, x_valid),
                                       verbose=verb)
        time_train = time() - tic
        
        # Assure the models are resuming from the best models
        if path_temp_best is not None:
            self.autoencoder = keras.models.load_model(chkpt)
            self.encoder = self.autoencoder.layers[1]
            self.decoder = self.autoencoder.layers[2]
        
        return history, time_train
    
        
    def save(self, path_folder):
        """
        Deprecated! Because the availability for saving/loading model is limited.
        The method is to save the models of autoencoders as several hdf5 files in a given path of the folder.
        Args:
            path_folder (string): the given path of the folder where contains encoder, decoder, and autoencoder
        Returns:
            msg (string): the message for the saving process
        """
        # Create the message for saving model
        msg = ""
        if os.path.exists(path_folder):
            msg += "There is a existing folder.\r\n"
        else:
            os.makedirs(path_folder)
            msg += "Create a new folder.\r\n"
        
        # Save the models of encoder and decoder with save()
        self.encoder.compile(optimizer="adam", loss="binary_crossentropy") # avoid saving warning
        self.decoder.compile(optimizer="adam", loss="binary_crossentropy") # avoid saving warning
        self.encoder.save(path_folder+"/encoder.h5")
        self.decoder.save(path_folder+"/decoder.h5")
        
        # Save the model of autoencoder with json and save_weights(). 
        # Because autoencoder contains special loss function and sample function.
        self.autoencoder.save_weights(path_folder+"/weightAutoencoder.h5")
        with open(path_folder+"/configAutoencoder.json", "w") as json_file:
            json_file.write(self.autoencoder.to_json())
        msg += "successful.\r\n"
        return msg

# Convolutional autoencoder

In [5]:
class ConvAE(AE):
    
    def __init__(self, 
                 dim_input, dim_latent=2,
                 lay_conv_enc=[8, 32], lay_conv_dec=None, size_kernel=3, strides=2, act_conv="leaky_relu", padding="same",
                 lay_den_enc=[64], lay_den_dec=None, act_dense="leaky_relu", dropout_dense=0.5,
                 batch_norm = True):
        """
        The basic properties and pipeline will be defined in the initialization.
        The dimension of input should be a form of a picture presented by a list [width, height, channels].
        It should be noted that lay_den_enc defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16] and plus the latent 2 means that the nodes of encoder is [64, 16, 2]. 
        Meanwhile decoder will be [2, 16, 64]. 
        It is similar for lay_conv_enc but decoder is not purely symmetric for convolution layers for this version.
        Args:
            dim_input (list[int]): the dimension of input. E.g. [32, 28, 3] means 32 by 28 RGB pixels.
            dim_latent (in): the number of the dimension for latent feature. Default is 2.
            lay_conv_enc (list[int]): the numbers of each convolution layer of encoder. Default is [8, 32].
            lay_conv_dec (list[int]): the numbers of each convolution layer of decoder. Default is None that means reverse order of encoder.
            size_kernel (int): the size of filter kernel. Default 3 means 3 by 3.
            strides (int): the stride for convolution. Default is 2.
            act_conv (string): the activation function of each convolution layer. Default is "leaky_relu".
            padding (string): the padding method for convolution. Default is "same".
            lay_den_enc (list[int]): the numbers of each dense layer of encoder. Default is [64].
            lay_den_dec (list[int]): the numbers of each dense layer of decoder. Default is None that means reverse order of encoder.
            act_dense (string): the activation function of each dense layer. Default is "leaky_relu".
            dropout_dense (float): the dropout layer. Default is 0.5.
            batch_norm (bool): determine if apply batch normalization after dense/conv layers. Default is True and epsilon=1e-5.
        """
        
        # Initialize some setting 
        self._dim_input = dim_input # dim_input is (width, height, channels)
        self._inputs = Input(shape=(dim_input)) 
        self._dim_latent = dim_latent
        if lay_conv_dec is None:
            lay_conv_dec = lay_conv_enc[::-1]
        if lay_den_dec is None:
            lay_den_dec = lay_den_enc[::-1]

        print(lay_den_dec)
            
        self._encoding(lay_conv_enc, size_kernel, strides, act_conv, padding,
                       lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        self._decoding(lay_conv_dec, size_kernel, strides, act_conv, padding,
                       lay_den_dec, batch_norm, act_dense, dropout_dense)
        
        self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)), name="autoencoder")

        
    def _stack_conv(self, x, 
                   lay_conv_enc, size_kernel, strides, padding,
                   batch_norm, act_conv, is_trans=False):
        """
        Stacking for convolutional layers whether encoder or decoder.
        Transpose convolutional layers are applied for decoder.
        The sequence is:
            Convolution(Transpose) layers
            Batch normalization
            Activation function
        """
        for filters in lay_conv_enc:
            if is_trans:
                x = Conv2DTranspose(filters=filters,
                                    kernel_size=size_kernel,
                                    strides=strides,
                                    padding=padding)(x)
            else:
                x = Conv2D(filters=filters,
                           kernel_size=size_kernel,
                           strides=strides,
                           padding=padding)(x)
            if batch_norm :
                x = BatchNormalization(epsilon=1e-5)(x)
            if act_conv == "leaky_relu":
                x = LeakyReLU()(x)
            else:
                x = Activation(act_conv)(x)
        return x

    
    def _encoding(self,
                  lay_conv_enc, size_kernel, strides, act_conv, padding,
                  lay_den_enc, batch_norm, act_dense, dropout_dense):
        """ 
        The flow of encoding
        """
        x = self._inputs
        
        # Stack of Conv2D layers
        x = self._stack_conv(x, 
                            lay_conv_enc, size_kernel, strides, padding,
                            batch_norm, act_conv)

        # Shape info needed to build Decoder Model
        self._shape_last_conv = K.int_shape(x)

        # Stack of Dense layers
        x = Flatten()(x)
        x = self._stack_dense(x, lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        # Construct the latent as the output and build the encorder pipeline
        z = Dense(self._dim_latent)(x)
        self.encoder = Model(self._inputs, z, name="encoder")

        
    def _decoding(self,
                  lay_conv_dec, size_kernel, strides, act_conv, padding,
                  lay_den_dec, batch_norm, act_dense, dropout_dense):
        """ 
        The flow of decoding
        """
        shape_last_conv = self._shape_last_conv
        # Build the Decoder Model
        input_latent = Input(shape=(self._dim_latent,), name="decoder_input")
        x = input_latent
        
        # Stack of Dense layers
        x = self._stack_dense(x, lay_den_dec, batch_norm, act_dense, dropout_dense)
        x = Dense(shape_last_conv[1] * shape_last_conv[2] * shape_last_conv[3])(x)
        x = Reshape((shape_last_conv[1], shape_last_conv[2], shape_last_conv[3]))(x)

        # Stack of Transposed Conv2D layers
        x = self._stack_conv(x, 
                            lay_conv_dec, size_kernel, strides, padding,
                            batch_norm, act_conv, is_trans=True)

        # Build the Conv2DTranspose layer for the pixel dimension
        x = Conv2DTranspose(filters=self._dim_input[-1],
                            kernel_size=size_kernel,
#                             strides=strides,
                            padding=padding)(x)

        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Activation("sigmoid", name="decoder_output")(x)
        self.decoder = Model(input_latent, outputs, name="decoder")

# Variational autoencoder

In [5]:
class VAE(AE):
    _std_eps = 1.0
    
    def __init__(self, 
                 dim_input, dim_latent=2,
                 lay_den_enc=[64], lay_den_dec=None, act_dense="leaky_relu", dropout_dense=0.5,
                 batch_norm=True,
                 rat_recon=0.998):
        """
        The basic properties and pipeline will be defined in the initialization.
        It should be noted that lay_den_enc defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16] and plus the latent 2 means that the nodes of encoder is [64, 16, 2]. 
        Meanwhile decoder will be [2, 16, 64].  
        There is another parameter should noted that rat_recon=0.5 doesn't mean the effect is half.
        Because KL loss and reconstruction loss are not the same scale.
        Args:
            dim_input (int): the number of input dimension. All features are flatten as a vector.
            dim_latent (in): the number of the dimension for latent feature. Default is 2.
            lay_den_enc (list[int]): the numbers of each dense layer. Default is [64].
            lay_den_dec (list[int]): the numbers of each dense layer of decoder. Default is None that means reverse order of encoder.
            act_dense (string): the activation function. Default is "leaky_relu".
            dropout_dense (float): the dropout layer. Default is 0.5.
            batch_norm (bool): determine if apply batch normalization after dense layers. Default is True and epsilon=1e-5.
            rat_recon (float): the parameter for tuning the effects between KL loss and reconstruction loss.
        """
        
        # Initialize some setting 
        super().__init__(dim_input=dim_input, dim_latent=dim_latent,
                         lay_den_enc=lay_den_enc, lay_den_dec=lay_den_dec, act_dense=act_dense, dropout_dense=dropout_dense,
                         batch_norm=batch_norm)
        self._rat_recon = rat_recon
        
        
    def _encoding(self, lay_den_enc, batch_norm, act_dense, dropout_dense):
        """ 
        The flow of encoding with the layers of mean and variance.
        """
        dim_latent = self._dim_latent
        x = self._inputs

        # Stack of Dense layers
        x = self._stack_dense(x, lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        # Build the mean and variance layers
        self._z_mean = Dense(dim_latent)(x)
        self._z_sigma_log = Dense(dim_latent)(x) # log for linear dense

        # Define the sampling function for the sampling layer.
        # Note that the function must be in the same location with encoding for saving/loadind model.
        def sampling(args, std_eps):
            z_mean, z_sigma_log = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], K.shape(z_mean)[1]),
                                      mean=0., stddev=std_eps)
            return z_mean + K.exp(z_sigma_log) * epsilon  
        
        # Construct the latent as the output and build the encorder pipeline
        z = Lambda(sampling, arguments={"std_eps":self._std_eps})([self._z_mean, self._z_sigma_log])
        self.encoder = Model(self._inputs, z, name="encoder")

        
    def _loss_vae(self, tensor_input, tensor_decode):
        """ """
        z_mean = self._z_mean
        z_sigma_log = self._z_sigma_log
        rat_recon = self._rat_recon
        
        lossRecon =  binary_crossentropy(K.flatten(tensor_input), K.flatten(tensor_decode))
#         lossRecon =  mean_squared_error(K.flatten(tensor_input), K.flatten(tensor_decode))
        lossKL = - 0.5 * K.sum(1 + 2 * z_sigma_log - K.square(z_mean) - K.square(K.exp(z_sigma_log)), axis=-1)
        return rat_recon * lossRecon + (1 - rat_recon) * lossKL
        
        
    def fit(self,
            x_train, x_valid,
            num_epochs=50, size_batch=32, name_optim="adam", metrics=None, verb=1,
            path_temp_best=None, patience=3):
        """
        The method is for training process. 
        The users can call this method easily just putting training and validation datasets.
        The dimension of dataset is determined by [#instance, *dimInput].
        For example, dimInput is flatten as a number and the dimension of dataset is [#instance, #feature]
        If dimInput is a list to represent [width, height, channels], the dimension of dataset is [#instance, width, height, channels]
        Args:
            x_train (numpy ndarray): the training dataset.
            x_valid (numpy ndarray): the validation dataset.
            num_epochs (int): the maximal epochs for training. Default is 50.
            size_batch (int): the batch size. Default is 32.
            name_optim (string): the method for optimization. Default is adam.
            metrics (list(string or keras metrics)): the usage is the same with keras metrics for compile 
            verb(int): verbose; it is applied for both of fit and callback. The setting is similar to keras.
            path_temp_best (string): the temperory path of the best model for early-stop. Default None means without early-stop. 
            patience (int): the times of epochs to allow further trying if current loss is not better than the best. 
        Returns:
            history (keras.callbacks.History): the learning curving for the training process
            time_train (float): the consuming time of the training 
        """
        self.autoencoder.compile(optimizer=name_optim, loss=self._loss_vae, metrics=metrics)
        
        if path_temp_best is None:
            callbacks = None
        else:
            if not os.path.exists(path_temp_best): # make sure the folder exists
                os.makedirs(path_temp_best)
            
            name_temp = "AutoEncoder" + str(time()) # use timestamp as unique name
            cb_es = EarlyStopping(monitor="val_loss", patience=patience, verbose=verb, mode="auto")
            chkpt = path_temp_best + "/" + name_temp + ".hdf5"
            cb_cp = ModelCheckpoint(filepath=chkpt, monitor="val_loss", verbose=verb, save_best_only=True, mode="auto")
            callbacks = [cb_es, cb_cp]
        
        # Train the autoencoder
        tic = time()
        history = self.autoencoder.fit(x_train, x_train,
                                       epochs=num_epochs,
                                       batch_size=size_batch, shuffle=True,
                                       callbacks=callbacks,
                                       validation_data=(x_valid, x_valid),
                                       verbose=verb)
        time_train = time() - tic
        
        # Assure the models are resuming from the best models
        if path_temp_best is not None:
            self.autoencoder = keras.models.load_model(chkpt, custom_objects={"_loss_vae": self._loss_vae})
            self.encoder = self.autoencoder.layers[1]
            self.decoder = self.autoencoder.layers[2]
        
        return history, time_train

# Convolutinal VAE

In [6]:
class ConvVAE(VAE):
    
    def __init__(self, 
                 dim_input, dim_latent=2,
                 lay_conv_enc=[8, 32], lay_conv_dec=None, size_kernel=3, strides=2, act_conv="leaky_relu", padding="same",
                 lay_den_enc=[64], lay_den_dec=None, act_dense="leaky_relu", dropout_dense=0.5,
                 batch_norm=True,
                 rat_recon=0.998):
        """
        The basic properties and pipeline will be defined in the initialization.
        The dimension of input should be a form of a picture presented by a list [width, height, channels].
        It should be noted that lay_den_enc defines the first half(encoder) of network. 
        The decoder will be reflected structure.
        For example, [64, 16] and plus the latent 2 means that the nodes of encoder is [64, 16, 2]. 
        Meanwhile decoder will be [2, 16, 64].
        It is similar for lay_conv_enc but decoder is not purely symmetric for convolution layers for this version.
        There is another parameter should noted that rat_recon=0.5 doesn't mean the effect is half.
        Because KL loss and reconstruction loss are not the same scale.
        Args:
            dim_input (list[int]): the dimension of input. E.g. [32, 28, 3] means 32 by 28 RGB pixels.
            dim_latent (in): the number of the dimension for latent feature. Default is 2.
            lay_conv_enc (list[int]): the numbers of each convolution layer. Default is [8, 32].
            lay_conv_dec (list[int]): the numbers of each convolution layer of decoder. Default is None that means reverse order of encoder.
            size_kernel (int): the size of filter kernel. Default 3 means 3 by 3.
            strides (int): the stride for convolution. Default is 2.
            act_conv (string): the activation function of each convolution layer. Default is "leaky_relu".
            padding (string): the padding method for convolution. Default is "same".
            lay_den_enc (list[int]): the numbers of each dense layer. Default is [64, 2].
            lay_den_dec (list[int]): the numbers of each dense layer of decoder. Default is None that means reverse order of encoder.
            act_dense (string): the activation function of each dense layer. Default is "leaky_relu".
            dropout_dense (float): the dropout layer. Default is 0.5.
            batch_norm (bool): determine if apply batch normalization after dense/conv layers. Default is True and epsilon=1e-5.
            rat_recon (float): the parameter for tuning the effects between KL loss and reconstruction loss.
        """        
        # Initialize some setting 
        self._dim_input = dim_input # dim_input is (width, height, channels)
        self._inputs = Input(shape=(dim_input)) 
        self._dim_latent = dim_latent
        if lay_conv_dec is None:
            lay_conv_dec = lay_conv_enc[::-1]
        if lay_den_dec is None:
            lay_den_dec = lay_den_enc[::-1]
        
        self._encoding(lay_conv_enc, size_kernel, strides, act_conv, padding,
                       lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        self._decoding(lay_conv_dec, size_kernel, strides, act_conv, padding,
                       lay_den_dec, batch_norm, act_dense, dropout_dense)
        
        self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)), name="autoencoder")
        self._rat_recon = rat_recon
        
    
    def _stack_conv(self, x, 
                   lay_conv_enc, size_kernel, strides, padding,
                   batch_norm, act_conv, is_trans=False):
        """
        Stacking for convolutional layers whether encoder or decoder. 
        It's basic the same with the function of ConvAE.
        """
        for filters in lay_conv_enc:
            if is_trans:
                x = Conv2DTranspose(filters=filters,
                                    kernel_size=size_kernel,
                                    strides=strides,
                                    padding=padding)(x)
            else:
                x = Conv2D(filters=filters,
                           kernel_size=size_kernel,
                           strides=strides,
                           padding=padding)(x)
            if batch_norm :
                x = BatchNormalization(epsilon=1e-5)(x)
            if act_conv == "leaky_relu":
                x = LeakyReLU()(x)
            else:
                x = Activation(act_conv)(x)
        return x

        
    def _encoding(self,
                  lay_conv_enc, size_kernel, strides, act_conv, padding,
                  lay_den_enc, batch_norm, act_dense, dropout_dense):
        """ """
        dim_latent = self._dim_latent
        x = self._inputs
        
        # Stack of Conv2D layers
        x = self._stack_conv(x, 
                            lay_conv_enc, size_kernel, strides, padding,
                            batch_norm, act_conv)

        # Shape info needed to build Decoder Model
        self._shape_last_conv = K.int_shape(x)

        # Stack of Dense layers
        x = Flatten()(x)
        x = self._stack_dense(x, lay_den_enc, batch_norm, act_dense, dropout_dense)
        
        # Build the mean and variance layers
        self._z_mean = Dense(dim_latent)(x)
        self._z_sigma_log = Dense(dim_latent)(x) # log for linear dense

        # Define the sampling function for the sampling layer.
        # Note that the function must be in the same location with encoding for saving/loadind model.
        def sampling(args, std_eps):
            z_mean, z_sigma_log = args
            epsilon = K.random_normal(shape=(K.shape(z_mean)[0], K.shape(z_mean)[1]),
                                      mean=0., stddev=std_eps)
            return z_mean + K.exp(z_sigma_log) * epsilon  
        
        # Construct the latent as the output and build the encorder pipeline
        z = Lambda(sampling, arguments={"std_eps":self._std_eps})([self._z_mean, self._z_sigma_log])
        self.encoder = Model(self._inputs, z, name="encoder")

        
    def _decoding(self,
                  lay_conv_dec, size_kernel, strides, act_conv, padding,
                  lay_den_dec, batch_norm, act_dense, dropout_dense):
        """ """
        shape_last_conv = self._shape_last_conv
        # Build the Decoder Model
        input_latent = Input(shape=(self._dim_latent,), name="decoder_input")
        x = input_latent
        
        # Stack of Dense layers
        x = self._stack_dense(x, lay_den_dec, batch_norm, act_dense, dropout_dense)
        x = Dense(shape_last_conv[1] * shape_last_conv[2] * shape_last_conv[3])(x)
        x = Reshape((shape_last_conv[1], shape_last_conv[2], shape_last_conv[3]))(x)

        # Stack of Transposed Conv2D layers
        x = self._stack_conv(x, 
                             lay_conv_dec, size_kernel, strides, padding,
                             batch_norm, act_conv, is_trans=True)

        # Build the Conv2DTranspose layer for the pixel dimension
        x = Conv2DTranspose(filters=self._dim_input[-1],
                            kernel_size=size_kernel,
#                             strides=strides,
                            padding=padding)(x)

        # Reconstruct the pixels as the output and build the decorder pipeline
        outputs = Activation("sigmoid", name="decoder_output")(x)
        self.decoder = Model(input_latent, outputs, name="decoder")